In [6]:
from konlpy.tag import Okt
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np
import warnings
from nltk.corpus import stopwords

warnings.filterwarnings(action='ignore')

In [7]:
title='zepeto.csv'
word_list =set(['이','개','저','요','제발','뭐','하나','그냥','해','데','못','때문','그것','게','지금','내','걸','나','위','로','캐','이안','고','처럼','밑','스','방금','제','다시','왜','안','갑자기','처음','전','줄','한번','오늘','아예','해도','원래','번','만','하라','때','그거','심해','가끔','피하','하나요','욤','튕기','뭘','약간','렉좀','인','옆','것','그','또','꼭','명','건','부분','자주','글','수가','분','곳','경우','바로','자꾸','용','여','터','오','튕겨','어쩌','더','수','조금','짱','음','등','해주시','함','등등','찍','일단','자기','생기','일리','후','뒤','이번','이후','가기','중이','오지','땜','응','장공','보고','너','듯','세','배','무엇','게속','안나','그레','전혀','다가','놀수','정말','별로','굿','벌','색','제패','토','새','쫌','염','것좀','강추','멩','정도','번','기','거의','살','달','좀','거','진짜','저기','밌','이나','주','안눌려','재','완전','최고','이건','임','애','막','슴','개도','주기도','여기','무','영','고요','니','영','아주','생각','생','맘','구','산','버','화가','혐기','주시','물','피','다른','리지','계속','무슨','잘못','뭔가','실망','욤','난','답','일이','자체','끼리','다음','습','계속','가요','보','최악','잠깐','이상','몇번','며칠','어디','가지','직접','눈','업','날','끝','뎅','하니','혹시','건가','네','요즘','도','를','이제','좋더','얼마','언제','대박','거기','된거','그대로','주심','예전','제대로','분명','시','제페토'])
word_list

{'가기',
 '가끔',
 '가요',
 '가지',
 '갑자기',
 '강추',
 '개',
 '개도',
 '거',
 '거기',
 '거의',
 '건',
 '건가',
 '걸',
 '것',
 '것좀',
 '게',
 '게속',
 '경우',
 '계속',
 '고',
 '고요',
 '곳',
 '구',
 '굿',
 '그',
 '그거',
 '그것',
 '그냥',
 '그대로',
 '그레',
 '글',
 '기',
 '꼭',
 '끝',
 '끼리',
 '나',
 '난',
 '날',
 '내',
 '너',
 '네',
 '놀수',
 '눈',
 '니',
 '다가',
 '다른',
 '다시',
 '다음',
 '달',
 '답',
 '대박',
 '더',
 '데',
 '뎅',
 '도',
 '된거',
 '뒤',
 '듯',
 '등',
 '등등',
 '때',
 '때문',
 '땜',
 '또',
 '렉좀',
 '로',
 '를',
 '리지',
 '막',
 '만',
 '맘',
 '멩',
 '며칠',
 '명',
 '몇번',
 '못',
 '무',
 '무슨',
 '무엇',
 '물',
 '뭐',
 '뭔가',
 '뭘',
 '밌',
 '밑',
 '바로',
 '방금',
 '배',
 '버',
 '번',
 '벌',
 '별로',
 '보',
 '보고',
 '부분',
 '분',
 '분명',
 '산',
 '살',
 '새',
 '색',
 '생',
 '생각',
 '생기',
 '세',
 '수',
 '수가',
 '스',
 '슴',
 '습',
 '시',
 '실망',
 '심해',
 '아예',
 '아주',
 '안',
 '안나',
 '안눌려',
 '애',
 '약간',
 '어디',
 '어쩌',
 '언제',
 '얼마',
 '업',
 '여',
 '여기',
 '염',
 '영',
 '옆',
 '예전',
 '오',
 '오늘',
 '오지',
 '완전',
 '왜',
 '요',
 '요즘',
 '욤',
 '용',
 '원래',
 '위',
 '음',
 '응',
 '이',
 '이건',
 '이나',
 '이번',
 '이상',
 '이안',
 '이제',
 '이후',
 '인',
 '

In [8]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [9]:
def get_nouns(tokenizer, sentence):
    nouns=tokenizer.nouns(sentence)
    for word in word_list: 
        nouns=remove_values_from_list(nouns,word)
    return nouns

def tokenize(df):
    tokenizer = Okt()
    processed_data = [] 
    for sent in tqdm(df['content']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data

In [10]:
def save_processed_data(processed_data):
    with open("output/tokenized_data_"+title, 'w', newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        for data in processed_data:
            writer.writerow(data)
            
if __name__ == '__main__':
    tdf = pd.read_csv("data/zepeto.csv")
    df=tdf[['userName','at','content','score','thumbsUpCount']]
    df.dropna(subset = ['content'])
    processed_data = tokenize(df)
    save_processed_data(processed_data)

100%|███████████████████████████████████████████████████████████████████████████| 52550/52550 [01:55<00:00, 453.60it/s]


In [11]:
processed_data = [sent.strip().split(",") for sent in tqdm(open("output/tokenized_data_"+title,'r',encoding='utf-8').readlines())]
processed_data = DataFrame(processed_data)
processed_data[0] = processed_data[0].replace("", np.nan)
processed_data = processed_data[processed_data[0].notnull()]
processed_data = processed_data.values.tolist()
processed_data2=[]
for i in processed_data:
    i = list(filter(None, i))
    processed_data2.append(i)
processed_data = processed_data2
processed_data

100%|███████████████████████████████████████████████████████████████████████| 52550/52550 [00:00<00:00, 1501067.01it/s]


[['사용성', '개선', '오히려', '월드', '기능', '마비', '사용성', '호환', '문제'],
 ['퀘스트',
  '스크래치',
  '광고',
  '퀘스트',
  '카드',
  '완료',
  '방치',
  '일',
  '광고',
  '콜라보',
  '정신',
  '유저',
  '속눈썹',
  '착용',
  '모자',
  '파트',
  '모자',
  '머리핀',
  '해결',
  '중복',
  '착용',
  '앞머리',
  '착용',
  '붙임머리',
  '포니테일',
  '선택',
  '붙임머리',
  '앞머리',
  '선택',
  '문제',
  '아래',
  '문제',
  '은',
  '중복',
  '착용',
  '허용',
  '해결'],
 ['체제', '읍'],
 ['캐릭터',
  '소통',
  '단점',
  '첫',
  '자신',
  '캐릭터',
  '사진',
  '있었르',
  '현질',
  '면서',
  '캐릭터',
  '추가',
  '이유',
  '남',
  '캐릭터',
  '사진',
  '젬',
  '꽤',
  '레벨',
  '플레이',
  '젬',
  '기분',
  '나머지'],
 ['케릭', '키', '조절', '예', '식', '얼굴', '몸', '팔다리', '신체', '조절'],
 ['스크래치',
  '기분',
  '앱',
  '중단',
  '젠',
  '접속',
  '피해',
  '피해',
  '신고',
  '접지',
  '최대한',
  '점',
  '별',
  '별'],
 ['게임'],
 ['업데이트',
  '업데이트',
  '월드',
  '방',
  '검색',
  '기능',
  '방',
  '검색',
  '기능',
  '월드',
  '갈수',
  '그게',
  '재미'],
 ['퍼즐', '스파이', '미션', '잼'],
 ['아유'],
 ['아이', '세면', '요별'],
 ['동영상', '게시', '동영상', '동영상', '업로드', '동영상', '길이'],
 ['마디',
  '코인',
  '출석',
  '젬',
  '노

In [12]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [13]:
dictionary = corpora.Dictionary(processed_data)
dictionary.filter_extremes(no_below=2, no_above=0.5)

2022-06-17 10:51:13,218 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-06-17 10:51:13,292 : INFO : adding document #10000 to Dictionary(7291 unique tokens: ['개선', '기능', '마비', '문제', '사용성']...)
2022-06-17 10:51:13,360 : INFO : adding document #20000 to Dictionary(10978 unique tokens: ['개선', '기능', '마비', '문제', '사용성']...)
2022-06-17 10:51:13,414 : INFO : adding document #30000 to Dictionary(13369 unique tokens: ['개선', '기능', '마비', '문제', '사용성']...)
2022-06-17 10:51:13,467 : INFO : adding document #40000 to Dictionary(15412 unique tokens: ['개선', '기능', '마비', '문제', '사용성']...)
2022-06-17 10:51:13,489 : INFO : built Dictionary(15985 unique tokens: ['개선', '기능', '마비', '문제', '사용성']...) from 44067 documents (total 284936 corpus positions)
2022-06-17 10:51:13,489 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(15985 unique tokens: ['개선', '기능', '마비', '문제', '사용성']...) from 44067 documents (total 284936 corpus positions)", 'datetime': '2022-06-17T10:51:13.489110', 'gensi

In [14]:
corpus = [dictionary.doc2bow(text) for text in processed_data]

In [15]:
# -*- coding: utf-8 -*-
from konlpy.tag import Mecab
from tqdm import tqdm
import re
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
import logging
import pickle
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

coherence_values = []
model_list = []

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


def find_optimal_number_of_topics(dictionary, corpus, processed_data):
    limit = 40;
    start = 2;
    step = 1;

    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step)

    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

if __name__ == '__main__':
    
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # 최적의 토픽 수 찾기
    #find_optimal_number_of_topics(dictionary, corpus, processed_data)

Number of unique tokens: 7057
Number of documents: 44067


In [16]:
#j=2
#for i in range(len(coherence_values)+1):
#    print(j,":",coherence_values[i])
#    j+=1

In [17]:
num_topics = 6
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every)

2022-06-17 10:51:13,698 : INFO : using autotuned alpha, starting with [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]
2022-06-17 10:51:13,699 : INFO : using serial LDA version on this node
2022-06-17 10:51:13,703 : INFO : running online (multi-pass) LDA training, 6 topics, 20 passes over the supplied corpus of 44067 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-06-17 10:51:13,704 : INFO : PROGRESS: pass 0, at document #2000/44067
2022-06-17 10:51:14,122 : INFO : optimized alpha [0.14284682, 0.13257924, 0.12983096, 0.13743171, 0.12752636, 0.1195555]
2022-06-17 10:51:14,122 : INFO : merging changes from 2000 documents into a model of 44067 documents
2022-06-17 10:51:14,128 : INFO : topic #5 (0.120): 0.029*"아이템" + 0.022*"기능" + 0.021*"게임" + 0.017*"계정" + 0.017*"크리에이터" + 0.017*"선물" + 0.014*"로그인" + 0.014*"스텀" + 0.013*"돈" + 0.012*"광고"
2022-06-17 10:51:14,128 : I

2022-06-17 10:51:15,528 : INFO : merging changes from 2000 documents into a model of 44067 documents
2022-06-17 10:51:15,530 : INFO : topic #2 (0.111): 0.027*"겜" + 0.025*"사람" + 0.025*"점프" + 0.022*"맵" + 0.021*"젬" + 0.019*"신고" + 0.019*"단점" + 0.017*"유저" + 0.012*"옷" + 0.011*"계정"
2022-06-17 10:51:15,530 : INFO : topic #5 (0.118): 0.070*"로그인" + 0.057*"계정" + 0.027*"선물" + 0.022*"기능" + 0.016*"재미" + 0.015*"게임" + 0.015*"크리에이터" + 0.014*"아이템" + 0.011*"동의" + 0.010*"비밀번호"
2022-06-17 10:51:15,531 : INFO : topic #1 (0.136): 0.096*"게임" + 0.041*"렉" + 0.032*"월드" + 0.031*"업데이트" + 0.027*"친구" + 0.023*"랙" + 0.021*"옷" + 0.016*"점" + 0.015*"오류" + 0.014*"폰"
2022-06-17 10:51:15,531 : INFO : topic #3 (0.156): 0.055*"현질" + 0.039*"옷" + 0.038*"렉" + 0.033*"캐릭터" + 0.029*"젬" + 0.027*"유도" + 0.023*"게임" + 0.018*"점" + 0.017*"코인" + 0.016*"유저"
2022-06-17 10:51:15,532 : INFO : topic #0 (0.178): 0.133*"젬" + 0.043*"옷" + 0.037*"현질" + 0.035*"게임" + 0.034*"코인" + 0.025*"잼" + 0.020*"돈" + 0.018*"유저" + 0.012*"가격" + 0.012*"무료"
2022-06-17 

2022-06-17 10:51:16,568 : INFO : topic #0 (0.185): 0.135*"젬" + 0.047*"코인" + 0.043*"옷" + 0.032*"현질" + 0.027*"게임" + 0.025*"돈" + 0.025*"잼" + 0.015*"유저" + 0.011*"퀘스트" + 0.010*"점"
2022-06-17 10:51:16,568 : INFO : topic diff=0.259341, rho=0.277350
2022-06-17 10:51:16,571 : INFO : PROGRESS: pass 0, at document #28000/44067
2022-06-17 10:51:16,713 : INFO : optimized alpha [0.18628757, 0.17687124, 0.115564205, 0.16857499, 0.17682664, 0.122864]
2022-06-17 10:51:16,714 : INFO : merging changes from 2000 documents into a model of 44067 documents
2022-06-17 10:51:16,716 : INFO : topic #2 (0.116): 0.035*"사람" + 0.032*"신고" + 0.025*"유저" + 0.024*"겜" + 0.016*"맵" + 0.012*"변태" + 0.012*"말" + 0.010*"게시" + 0.009*"정지" + 0.008*"기능"
2022-06-17 10:51:16,717 : INFO : topic #5 (0.123): 0.066*"계정" + 0.047*"로그인" + 0.037*"선물" + 0.027*"기능" + 0.025*"재미" + 0.013*"비밀번호" + 0.013*"팔로우" + 0.012*"추가" + 0.012*"아이템" + 0.011*"크리에이터"
2022-06-17 10:51:16,717 : INFO : topic #4 (0.177): 0.195*"틱톡" + 0.054*"사람" + 0.052*"앱" + 0.046*"점

2022-06-17 10:51:17,590 : INFO : topic #5 (0.142): 0.089*"계정" + 0.058*"로그인" + 0.033*"초기" + 0.030*"이메일" + 0.027*"기능" + 0.023*"추가" + 0.022*"선물" + 0.021*"로그아웃" + 0.021*"삭제" + 0.019*"재미"
2022-06-17 10:51:17,591 : INFO : topic #3 (0.193): 0.085*"캐릭터" + 0.044*"옷" + 0.026*"사진" + 0.024*"얼굴" + 0.023*"추가" + 0.022*"옷장" + 0.018*"렉" + 0.012*"게임" + 0.012*"저장" + 0.012*"점"
2022-06-17 10:51:17,591 : INFO : topic #1 (0.206): 0.078*"업데이트" + 0.077*"게임" + 0.072*"렉" + 0.040*"월드" + 0.032*"친구" + 0.026*"오류" + 0.021*"폰" + 0.020*"랙" + 0.018*"업뎃" + 0.015*"꿀잼"
2022-06-17 10:51:17,591 : INFO : topic #0 (0.207): 0.075*"코인" + 0.060*"옷" + 0.058*"돈" + 0.035*"퀘스트" + 0.034*"젬" + 0.025*"게임" + 0.020*"광고" + 0.017*"현질" + 0.013*"가입" + 0.013*"잼"
2022-06-17 10:51:17,592 : INFO : topic diff=0.207003, rho=0.223607
2022-06-17 10:51:17,595 : INFO : PROGRESS: pass 0, at document #42000/44067
2022-06-17 10:51:17,734 : INFO : optimized alpha [0.2136157, 0.21525978, 0.12327453, 0.19788007, 0.18014711, 0.1454933]
2022-06-17 10:51:17,736

2022-06-17 10:51:18,393 : INFO : topic diff=0.195836, rho=0.203982
2022-06-17 10:51:18,396 : INFO : PROGRESS: pass 1, at document #8000/44067
2022-06-17 10:51:18,547 : INFO : optimized alpha [0.25612774, 0.24474922, 0.13204294, 0.21752055, 0.18215033, 0.15581855]
2022-06-17 10:51:18,553 : INFO : merging changes from 2000 documents into a model of 44067 documents
2022-06-17 10:51:18,555 : INFO : topic #2 (0.132): 0.029*"유저" + 0.027*"점프" + 0.026*"사람" + 0.024*"맵" + 0.023*"신고" + 0.023*"겜" + 0.014*"기능" + 0.013*"단점" + 0.011*"게시" + 0.010*"장점"
2022-06-17 10:51:18,555 : INFO : topic #5 (0.156): 0.078*"계정" + 0.061*"로그인" + 0.033*"기능" + 0.030*"이메일" + 0.029*"선물" + 0.025*"삭제" + 0.021*"재미" + 0.017*"초기" + 0.014*"로그아웃" + 0.012*"친구"
2022-06-17 10:51:18,556 : INFO : topic #3 (0.218): 0.085*"캐릭터" + 0.029*"얼굴" + 0.028*"옷" + 0.019*"저장" + 0.017*"추가" + 0.016*"점" + 0.015*"기능" + 0.014*"사진" + 0.014*"현질" + 0.013*"머리"
2022-06-17 10:51:18,556 : INFO : topic #1 (0.245): 0.087*"게임" + 0.072*"렉" + 0.041*"월드" + 0.037*"업

...

2022-06-17 10:51:56,636 : INFO : topic #5 (0.593): 0.107*"계정" + 0.070*"로그인" + 0.047*"삭제" + 0.046*"초기" + 0.035*"재미" + 0.033*"이메일" + 0.023*"로그아웃" + 0.018*"팔로우" + 0.016*"복구" + 0.016*"이름"
2022-06-17 10:51:56,637 : INFO : topic #4 (0.858): 0.121*"틱톡" + 0.089*"앱" + 0.077*"사람" + 0.031*"말" + 0.021*"인사" + 0.021*"점령" + 0.021*"영상" + 0.020*"욕" + 0.018*"설치" + 0.018*"박스"
2022-06-17 10:51:56,637 : INFO : topic #0 (1.323): 0.097*"옷" + 0.065*"코인" + 0.059*"젬" + 0.051*"돈" + 0.036*"광고" + 0.034*"현질" + 0.029*"퀘스트" + 0.024*"선물" + 0.015*"아이템" + 0.015*"잼"
2022-06-17 10:51:56,638 : INFO : topic #1 (1.509): 0.099*"게임" + 0.071*"렉" + 0.055*"업데이트" + 0.040*"월드" + 0.034*"점" + 0.033*"친구" + 0.029*"별" + 0.027*"오류" + 0.016*"랙" + 0.015*"폰"
2022-06-17 10:51:56,638 : INFO : topic diff=0.081869, rho=0.156110
2022-06-17 10:51:56,641 : INFO : PROGRESS: pass 18, at document #42000/44067
2022-06-17 10:51:56,717 : INFO : optimized alpha [1.3290888, 1.5234722, 0.47505984, 0.8585601, 0.8480522, 0.598696]
2022-06-17 10:51:56,719 : I

2022-06-17 10:51:57,081 : INFO : topic diff=0.092282, rho=0.154242
2022-06-17 10:51:57,085 : INFO : PROGRESS: pass 19, at document #8000/44067
2022-06-17 10:51:57,164 : INFO : optimized alpha [1.3724163, 1.5140893, 0.4688165, 0.8745512, 0.8003555, 0.59273607]
2022-06-17 10:51:57,166 : INFO : merging changes from 2000 documents into a model of 44067 documents
2022-06-17 10:51:57,168 : INFO : topic #2 (0.469): 0.042*"점프" + 0.035*"겜" + 0.033*"신고" + 0.014*"코드" + 0.014*"다운로드" + 0.011*"사랑" + 0.010*"만들기" + 0.010*"정지" + 0.009*"스트릿" + 0.009*"어플"
2022-06-17 10:51:57,168 : INFO : topic #5 (0.593): 0.094*"계정" + 0.073*"로그인" + 0.064*"삭제" + 0.037*"이메일" + 0.035*"재미" + 0.029*"초기" + 0.019*"팔로우" + 0.018*"로그아웃" + 0.015*"비밀번호" + 0.014*"메세지"
2022-06-17 10:51:57,168 : INFO : topic #3 (0.875): 0.101*"캐릭터" + 0.054*"기능" + 0.034*"추가" + 0.034*"얼굴" + 0.024*"사진" + 0.021*"저장" + 0.020*"사용" + 0.017*"머리" + 0.017*"스텀" + 0.016*"소통"
2022-06-17 10:51:57,169 : INFO : topic #0 (1.372): 0.107*"젬" + 0.070*"옷" + 0.054*"코인" + 0.

2022-06-17 10:51:57,688 : INFO : topic #4 (0.850): 0.118*"틱톡" + 0.097*"사람" + 0.076*"앱" + 0.045*"점령" + 0.037*"욕" + 0.034*"말" + 0.025*"유저" + 0.020*"영상" + 0.015*"인증" + 0.015*"설치"
2022-06-17 10:51:57,688 : INFO : topic #0 (1.412): 0.130*"젬" + 0.078*"옷" + 0.060*"현질" + 0.047*"코인" + 0.031*"광고" + 0.029*"돈" + 0.022*"유도" + 0.020*"선물" + 0.020*"잼" + 0.019*"아이템"
2022-06-17 10:51:57,688 : INFO : topic #1 (1.522): 0.120*"게임" + 0.074*"렉" + 0.048*"점" + 0.041*"월드" + 0.036*"별" + 0.027*"업데이트" + 0.027*"친구" + 0.021*"오류" + 0.018*"맵" + 0.017*"랙"
2022-06-17 10:51:57,689 : INFO : topic diff=0.106045, rho=0.154242
2022-06-17 10:51:57,692 : INFO : PROGRESS: pass 19, at document #22000/44067
2022-06-17 10:51:57,766 : INFO : optimized alpha [1.3808525, 1.5127679, 0.4726012, 0.82374763, 0.8695328, 0.5750123]
2022-06-17 10:51:57,768 : INFO : merging changes from 2000 documents into a model of 44067 documents
2022-06-17 10:51:57,770 : INFO : topic #2 (0.473): 0.048*"신고" + 0.045*"겜" + 0.031*"점프" + 0.016*"변태" + 0.015*"취

2022-06-17 10:51:58,252 : INFO : optimized alpha [1.3192725, 1.5450549, 0.47456434, 0.8221204, 0.882817, 0.5662437]
2022-06-17 10:51:58,262 : INFO : merging changes from 2000 documents into a model of 44067 documents
2022-06-17 10:51:58,263 : INFO : topic #2 (0.475): 0.036*"신고" + 0.035*"겜" + 0.017*"점프" + 0.015*"취" + 0.014*"변태" + 0.014*"다운로드" + 0.012*"어플" + 0.011*"관리" + 0.010*"정지" + 0.010*"만들기"
2022-06-17 10:51:58,264 : INFO : topic #5 (0.566): 0.098*"계정" + 0.059*"로그인" + 0.047*"재미" + 0.043*"삭제" + 0.025*"이메일" + 0.024*"초기" + 0.019*"비밀번호" + 0.017*"팔로우" + 0.013*"로그아웃" + 0.012*"메세지"
2022-06-17 10:51:58,264 : INFO : topic #4 (0.883): 0.171*"틱톡" + 0.085*"사람" + 0.080*"앱" + 0.032*"말" + 0.030*"점령" + 0.027*"욕" + 0.026*"영상" + 0.021*"유저" + 0.015*"설치" + 0.012*"제작자"
2022-06-17 10:51:58,265 : INFO : topic #0 (1.319): 0.094*"젬" + 0.089*"옷" + 0.052*"코인" + 0.046*"현질" + 0.038*"돈" + 0.037*"광고" + 0.029*"선물" + 0.022*"잼" + 0.017*"아이템" + 0.015*"유도"
2022-06-17 10:51:58,265 : INFO : topic #1 (1.545): 0.097*"게임" +

2022-06-17 10:51:58,690 : INFO : topic #0 (1.287): 0.097*"옷" + 0.080*"코인" + 0.063*"돈" + 0.054*"퀘스트" + 0.039*"젬" + 0.029*"광고" + 0.029*"현질" + 0.027*"선물" + 0.018*"아이템" + 0.017*"하루"
2022-06-17 10:51:58,690 : INFO : topic #1 (1.487): 0.102*"게임" + 0.074*"렉" + 0.042*"친구" + 0.040*"업데이트" + 0.034*"점" + 0.028*"오류" + 0.025*"월드" + 0.025*"로딩" + 0.024*"화면" + 0.020*"별"
2022-06-17 10:51:58,690 : INFO : topic diff=0.094992, rho=0.154242
2022-06-17 10:51:58,693 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel(num_terms=7057, num_topics=6, decay=0.5, chunksize=2000) in 44.99s', 'datetime': '2022-06-17T10:51:58.693603', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'created'}


In [18]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-06-17 10:51:58,706 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-06-17 10:51:58,710 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-06-17 10:51:58,713 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-06-17 10:51:58,716 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-06-17 10:51:58,719 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-06-17 10:51:58,721 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-06-17 10:51:58,725 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-06-17 10:51:58,728 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-06-17 10:51:58,731 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-06-17 10:51:58,735 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-06-17 10:51:58,738 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-06-17 10:51:58

Average topic coherence: -6.2194.
[([(0.101503834, '게임'),
   (0.07361908, '렉'),
   (0.04189555, '친구'),
   (0.04002369, '업데이트'),
   (0.033540934, '점'),
   (0.027698847, '오류'),
   (0.025252292, '월드'),
   (0.024956783, '로딩'),
   (0.02357197, '화면'),
   (0.020081498, '별'),
   (0.019416867, '폰'),
   (0.013667149, '시작'),
   (0.012298217, '시간'),
   (0.012123527, '랙'),
   (0.011789731, '수정'),
   (0.011713079, '플레이'),
   (0.010870788, '문제'),
   (0.010743937, '버튼'),
   (0.009743537, '추천'),
   (0.009718795, '접속')],
  -3.357244496725695),
 ([(0.097375244, '옷'),
   (0.079832874, '코인'),
   (0.06282561, '돈'),
   (0.053526573, '퀘스트'),
   (0.03879386, '젬'),
   (0.028984178, '광고'),
   (0.0289493, '현질'),
   (0.026798405, '선물'),
   (0.017895726, '아이템'),
   (0.016592517, '하루'),
   (0.010544605, '유도'),
   (0.010290191, '가입'),
   (0.009975911, '잼'),
   (0.009514092, '유저'),
   (0.009431035, '방법'),
   (0.009345402, '구매'),
   (0.008229144, '보상'),
   (0.0069874646, '가격'),
   (0.0066305464, '페북'),
   (0.0062320624

In [19]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [20]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'output/zepeto_topic_modeling_topic6_0616.html')